# **IMPLEMENTATION OF EDDYSED CODE**
This code is very closely mapped to the eddysed code of Ackerman and Marley from Original Fortran to my implemented Python Version.



### Import necessary python modules


In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### Global constants


In [6]:
""" This module defines physical and mathematical constants for atmospheric modeling. Careful for the units! """

MAXNZ = 50 # maximum number of layers
R_Gas = 8.3145e7 # Universal gas constant erg/(K mol)
StefBoltz = 5.6704e-5 # Stefan-Boltzmann constant erg/(cm^2 s K^4)
Avogadro = 6.02214179e23 # Avogadro's number molecules/mol
k_Boltz = 1.38064852e-16 # Boltzmann constant erg/K
Pi = np.pi

# Implementation of read_voyager.f
This code snippet talks about the way the input file from voyager_inputs in the main eddysed code has been handled.


In [20]:

def read_voyager(input_file):

    """
    The input file from voyager_inputs in the main eddysed code.
    reads the following inputs:
    Pressure 
    Temperature
    Gravity
    Effective Temperature
    Number of layers edges

    It calculates the number of layers , effective convective heat flux (chf), 
    molecular weight of atmosphere (assumed constant at 2.2 g/mol), and builds the T and P layers and sub layers along with the creation of altitude layers (Z) and its sub layers.


    and returns the following outputs:

    gravity, T_eff, num_layer_edges, p_layer_edges, t_layer_edges, z_layer_edges, num_layers,  p_layers, t_layers, z_layers, chf
    """

    input_data = []
    with open(input_file, "r") as f:
        lines = f.readlines()
        T_eff = float(lines[0])
        gravity = float(lines[1])
        num_layer_edges = int(lines[2])
        num_layers = num_layer_edges - 1
        for line in lines[4:]:
            parts = line.strip().split()
            input_data.append([float(x.replace("E", "e")) for x in parts])
    input_array = np.array(input_data)
    p_layer_edges = input_array[:, 0]
    t_layer_edges = input_array[:, 1]

    r_atmos = 8.3145e7/2.2

    z_layer_edges = np.zeros(num_layer_edges)
    z_layers = np.zeros(num_layers)
    p_layers = np.zeros(num_layers)
    t_layers = np.zeros(num_layers)

    for i in range(num_layers-1, -1, -1):
        itop = i
        ibot = i + 1
        dlnp = np.log(p_layer_edges[ibot]/p_layer_edges[itop])
        p_layers[i] = (p_layer_edges[itop] + p_layer_edges[ibot]) / 2
        scale_height = r_atmos * t_layer_edges[ibot] / gravity
        dz_layer = scale_height * dlnp
        z_layer_edges[i] = z_layers[ibot-1] + dz_layer
        dtdz = (t_layer_edges[itop] - t_layer_edges[ibot]) / dz_layer
        dz_pmid = scale_height * np.log(p_layer_edges[ibot]/p_layers[i])
        z_layers[i] = z_layer_edges[ibot] + dz_pmid
        t_layers[i] = t_layer_edges[ibot] + dtdz * dz_pmid

    chf = StefBoltz * T_eff**4 * np.ones(num_layers) #eqn 21 of Virga Paper where F_rad is assumed to be 0 in convective region

    return gravity, T_eff, num_layer_edges, p_layer_edges, t_layer_edges, z_layer_edges, num_layers,  p_layers, t_layers, z_layers, chf

# Implementation of pvap_gas.f
Here the pressure vapour pressure exerted by the condensate is found in the atmosphere of the planet.


In [ ]:
def pvap_gas(gas_name, t_layer, p_layer):
    """ 
    Here we are finding the e_s value for the clasius claperyon equation which is given in A1 eqn of Appendix A in AM01
    the input temperature is in Kelvin.
    """
    if gas_name == "NH3":
        pvap_nh3 = np.exp(-86596/t_layer**2 - 2161/t_layer +10.53) # in bars
        pvap_nh3 = pvap_nh3 * 1e6 # convert to dyne/cm^2
        return pvap_nh3
    else:
        raise ValueError("Gas not recognized in pvap_gas function.")

# Implementation of advdiff.f


In [16]:
def advdiff(qt, ad_qbelow, ad_qvs, ad_mixl, ad_dz, ad_fsed):

    ad_qc = max(0.0, qt - ad_qvs)
    advdiff = ad_qbelow * np.exp(-ad_fsed * ad_qc * ad_dz / (qt*ad_mixl)) - qt

    return ad_qc

# Implementation of find_root.f


In [ ]:
def find_root(f, y, params, xlow, xhigh, delta):

    max_iter = 50 # define maximum number of iterations
    epsilon = 1e-7 # define convergence criteria for independent variable
    
    # copy input range into local variables
    x1 = xlow
    x2 = xhigh

    # abort if root not bracketed by initial guesses
    f1 = f(x1, *params) - y
    f2 = f(x2, *params) - y

    # default return values
    xnew = x1
    fnew = f1

    if f1*f2 > 0:
        status = -1
        return xnew, status
    
    # iterate until root is found or maximum number of iterations are taken

    iter = 0
    while (np.abs(fnew) > delta) and (iter < max_iter) and (x2/x1 - 1 > epsilon):

        # estimate root from secant between endpoints
        slope = (f2-f1)/(x2-x1)
        xnew = x1 - f1/slope
        fnew = f(xnew, *params) - y

        # pick new x1 and x2 such that f(x) crosses zero between them
        if (fnew*f1 < 0) and (x2/xnew -1 > epsilon):
            x2 = xnew
            f2 = fnew
        else:
            x1 = xnew
            f1 = fnew
        
        iter += 1
    
    # Set status flag accordingly
    if (iter == max_iter) and (x2/x1 - 1 > epsilon):
        status = 1
    else:
        status = 0
    
    return xnew, status



# Implementation of calc_qc.f

This code snippet is about the implementation of the calc_qc function where the condensate mixing ratio is calculated and r_g and r_eff and number of droplet concentration is also calculated

This function requires to call the find_root, v_fall and advdiff to be called in to be executed properly.

In [ ]:
def calc_qc(gas_name, f_sed_layer, rho_p, mw_cloud, q_below, supsat, w_convect, mixl, dz_layer, gravity, mw_atmos, mfp, visc, t_layer, p_layer, sig_layer):

    """
    Calculate the condensate mixing ratio and the total mixing ratio along with the droplet effective and geometric mean radius along with coloumn droplet number concentration in a single atmospheric layer 
    
    Inputs :
    gas_name  :   name of condensing vapor
    f_sed_layer : rain factor for layer
    rho_p      :  density of condensed vapor (g/cm^3)
    mw_cloud  :  molecular weight of condensing vapor (g/mol)
    q_below    : total mixing ratio (vapor+condensate) below layer (g/g)
    supsat    :  fractional supersaturation persisting after condensation (assumed to be 0)
    w_convect   :convective velocity scale (cm/s)
    mixl       : convective mixing length scale (cm)
    dz_layer    :thickness of layer (cm)
    grav       : gravitational acceleration (cm/s^2)
    mw_atmos  :  molecular weight of atmosphere (g/mol)
    mfp  :       atmospheric mean free path (cm)
    visc :       atmospheric dynamic viscosity (dyne s/cm^2)
    t_layer  :   temperature of layer mid-pt (K)
    p_layer  :   air pressure (dyne/cm^2)
    sig_layer  : geometric std. dev. of lognormal size distribution


    Outputs :

    qc_layer :   condensate mixing ratio (g/g)
    qt_layer  :  gas + condensate mixing ratio (g/g)
    rg_layer :   geometric mean radius of condensate (cm)
    reff_layer : effective (area-weighted) radius of condensate (cm)
    ndz_layer  : column of particle concentration in layer (#/cm^2)
    qt_top   :   top of layer
    status_r  :  error status for finding rw
    status_q :   error status for finding qt
    
    
    
    We need to call the v_fall ,find_root and advdiff functions inside this function. to solve for qc, qt, rw, reff , rg and ndz.
    1. advdiff : to solve for qt at the top of the layer
    2. vfall : to solve for rw given w_convect
    3. find_root : to find roots of advdiff and vfall functions.
    4. using trapezoid rule to calculate layer averages of qt.
    5. diagnose qc from qt and qvs.
    6. calculate rg, reff and ndz using the diagnosed qc and rw.
    7. return all the calculated values.
    """

    pvap = pvap_gas(gas_name, t_layer, p_layer)
    fs = supsat + 1
    rho_atmos = p_layer / (R_Gas/mw_cloud * t_layer)
    qvs = fs * pvap / (R_Gas/mw_atmos * t_layer) / rho_atmos

    # Layer is cloud free
    if q_below <= qvs:

        qt_layer = q_below
        qt_top = q_below
        qc_layer = 0.0
        rg_layer = 0.0
        reff_layer = 0.0
        ndz_layer = 0

    # Layer has cloud
    else:
        
        # range of mixing ratios to search (g/g)
        qhigh = q_below
        qlow = qhigh * 0.001

        # precision of advective diffusive solution (g/g)
        delta_q = q_below /1000

        # load paramaters into advdiff common block
        ad_qbelow = q_below
        ad_qvs = qvs
        ad_mixl = mixl
        ad_dz = dz_layer
        ad_fsed = f_sed_layer
        params = 

        # Find total condensate mixing ratio at the top of the layer
        qt_top, status_q = find_root(advdiff, 0.0, qlow, qhigh, delta_q)

        # Use trapezoid rule to calculate layer averages
        # -- should integrate exponential
        qt_layer = 0.5 * (q_below + qt_top)

        # Diagnose condensate mixing ratio
        qc_layer = max(0.0, qt_layer - qvs)

        # Find rw corresponding to w_convect using function vfall()

        # load parameters into vfall common block
        vf_grav = gravity
        vf_mw_atmos = mw_atmos
        vf_mfp = mfp
        vf_visc = visc
        vf_p = p_layer
        vf_t = t_layer
        vf_rhop = rho_p

        # range of particle radii to search (cm)
        rlow = 1e-10
        rhigh = 10

        # precision of terminal velocity solution (cm/s)
        delta_v = w_convect / 1000
        rw_layer, status_r = find_root(vfall, rlow, rhigh, delta_v)

        # geometric std dev of lognormal size distribution
        lnsig2 = 0.5*np.log( sig_layer )**2

        # Compute exponent in vfall = w_convect r^alpha
        # sigma floor  for the purpose of alpha calculation
        sig_alpha = max(1.1, sig_layer)
        
        # Bulk of precip at r > rw: exponent between rw and rw*sig
        if f_sed_layer > 1:
            alpha = np.log(vfall(rw_layer * sig_alpha) / w_convect) / np.log(sig_alpha)
        
        # Bulk of precip at r < rw: exponent between rw/sig and rw
        else:
            alpha = np.log(w_convect / vfall(rw_layer / sig_alpha)) / np.log(sig_alpha)
        
        # geometric mean radius of lognormal size distribution
        rg_layer = f_sed_layer**(1/alpha) * rw_layer * np.exp(-(alpha+6)*lnsig2) #eqn 13 of AM01

        # droplet effective radius (cm)
        reff_layer = rg_layer * np.exp(5*lnsig2) #eqn 17 of AM01

        # Column droplet number concentration (cm^-2)
        ndz_layer = 3.0 * rho_atmos * qc_layer * dz_layer / (4.0 * Pi * rho_p * rg_layer**3 * np.exp(-9 * lnsig2))

    return qc_layer, qt_layer, rg_layer, reff_layer, ndz_layer, qt_top, status_r, status_q

# Implementation of layer.f

This code snippet works on the layer function of the code and calculates optical depth until a convergent solution to it is found.
This code snippet works on the layer function of the code and calculates optical depth until a convergent solution to it is found. The kz is also calculated using the Gierasch & Conrath formula over her itself

This function calls in the calc_qc function to be executed properly.


In [ ]:
def layer(nsub_max, gas_name, kz_min, cloudf_min, mw_cloud, mw_atmos, f_sed, rho_p, supsat, sig_layer, t_layer, p_layer, t_top, t_bot, p_top, p_bot, chf_layer, gravity, q_below, report_status_r, report_status_q):

    """ 
    Calculating layer condensate properties by iterating on optical depth until converges in one model layers (convergence on optical depth in sublayers)

    Inputs :
    nsub_max :   maximum number of sublayers for mesh refinement
    gas_name  :  name of condensing vapor
    kz_min :     minimum eddy diffusion coefficient (cm^2/s)
    cloudf_min :  minimum cloud fractional coverage
    mw_cloud  :  molecular weight of condensate (g/mol)
    rainf  :     rain factor
    rho_p  :     density of condensed vapor (g/cm^3)
    supsat  :    fractional supersaturation persisting after condensation
    sig_layer :   geometric std deviation of lognormal size distribution
    t_layer :    temperature at layer mid-pt (K)
    p_layer :    pressure at layer mid-pt (dyne/cm^2)
    t_top,bot :  temperature at top and bottom of layer (K)
    p_top,bot :  pressure at top and bottom of layer (dyne/cm^2)

    Outputs :
    qc_layer :   condensate mixing ratio (g/g)
    qt_layer  :  gas + condensate mixing ratio (g/g)
    rg_layer  :  geometric mean radius of condensate (cm)
    reff_layer : effective (area-weighted) radius of condensate (cm)
    ndz_layer  : column of particle concentration in layer (#/cm^2)
    opd_layer :  optical depth for conservative geometric scatterers

    report_status_r :   report error status for finding rw
    report_status_q :   report error status for finding qt

    This function calls calc_qc to calculate the condensate mixing ratio and other properties in the layer by subdividing the layer into sublayers and iterating until convergence on optical depth is achieved.



    """

    # Set error return codes to zero
    status_r = 0
    status_q = 0

    # Number of levels of grid refinement used 
    nsub = 1

    # diameter of atmospheric molecule (cm) (Rosner, 2000)
    # (3.711e-8 for air, 3.798e-8 for N2, 2.827e-8 for H2)
    d_molecule = 2.827e-8

    # parameter in Lennard-Jones potential for viscosity (K) (Rosner, 2000)
    # (78.6 for air, 71.4 for N2, 59.7 for H2)
    eps_k = 59.7

    # specific gas constant for atmosphere (erg/(g K))
    r_atmos = R_Gas / mw_atmos

    # specific heat of atmosphere (erg/K/g)
    c_p = 7./2. * r_atmos

    # pressure thickness of layer
    dp_layer = p_bot - p_top
    dlnp = np.log( p_bot/p_top )

    # temperature gradient 
    dtdlnp = ( t_top - t_bot ) / dlnp
    lapse_ratio = ( t_bot - t_top ) / dlnp / ( 2./7.*t_layer )

    # atmospheric density (g/cm^3)
    rho_atmos = p_layer / ( r_atmos * t_layer )

    # atmospheric scale height (cm)
    scale_h = r_atmos * t_layer / gravity #below eqn 5 in AM01

    # convective mixing length scale (cm): no less than 1/10 scale height
    mixl = max( 0.1, lapse_ratio ) * scale_h

    # mixing length = scale height matches Lunine (1989) model
    mixl = scale_h

    # scale factor for eddy diffusion: 1/3 is baseline
    scalef_kz = 1./3.

    # vertical eddy diffusion coefficient (cm^2/s)
    # from Gierasch and Conrath (1985)
    kz = scalef_kz * scale_h * (mixl/scale_h)**(4/3) *( ( r_atmos*chf_layer) / ( rho_atmos*c_p ) )**(1/3)#eqn 5 in AM01

    # no less than minimum value (for radiative regions)
    kz_layer = max( kz, kz_min )

    # convective velocity scale (cm/s)
    w_convect = kz_layer / mixl

    # cloud fractional coverage
    cloudf = cloudf_min + max( 0, min( 1, 1-lapse_ratio )) * ( 1 - cloudf_min )

    # atmospheric number density (molecules/cm^3)
    n_atmos = p_layer / ( k_Boltz*t_layer )

    # atmospheric mean free path (cm)
    mfp = 1. / ( np.sqrt(2.)*n_atmos*Pi*d_molecule**2 )

    # atmospheric viscosity (dyne s/cm^2)
    visc = 5./16.*np.sqrt( Pi*k_Boltz*t_layer*(mw_atmos/Avogadro)) / ( Pi*d_molecule**2 ) / ( 1.22 * ( t_layer / eps_k )**(-0.16) )
    # Eqn. B2 in Appendix B of AM01
    # --------------------------------------------------------------------
    # Top of convergence layer
    
    converge = False
    while not converge:
        
        # Zero cumulative values
        qc_layer = 0
        qt_layer = 0
        ndz_layer = 0
        opd_layer = 0

        # total mixing ratio and pressure at the bottom of sub layer
        qt_bot_sub = q_below
        p_bot_sub = p_bot

        # Loop over sub layers
        dp_sub = dp_layer / nsub
        for isub in range(nsub):

            qt_below = qt_bot_sub
            p_top_sub = p_bot_sub - dp_sub
            dz_sub = scale_h * np.log( p_bot_sub / p_top_sub )
            p_sub = ( p_bot_sub + p_top_sub ) / 2
            t_sub = t_bot + dtdlnp * np.log( p_bot / p_sub )

            # Calculate condensate mixing ratio etc. for sub layer
            qc_sub, qt_sub, rg_sub, reff_sub, ndz_sub, qt_top, status_r, status_q = calc_qc(gas_name, f_sed, rho_p, mw_cloud, qt_below, supsat, w_convect, mixl, dz_sub, gravity, mw_atmos, mfp, visc, t_sub, p_sub, sig_layer)

            # Vertical sums
            qc_layer += qc_sub * dp_sub/gravity
            qt_layer += qt_sub * dp_sub/gravity
            ndz_layer += ndz_sub

            if reff_sub > 0:
                opd_layer += 1.5 * qc_sub * dp_sub / gravity / (rho_p * reff_sub)
            
            # Increment values at the bottom of the sub layer
            qt_bot_sub = qt_top
            p_bot_sub = p_top_sub
        
            # Check convergence on optical depth
            if nsub_max > 1:
                converge = True
            elif nsub == 1:
                opd_test = opd_layer
            elif (opd_layer > 0) or (nsub >= nsub_max):
                converge = True
            elif np.abs(1 - opd_test/opd_layer) < 0.01:
                converge = True
            else:
                opd_test = opd_layer
            
            nsub = nsub * 2
        
        # --------------------------------------------------------------------
        # Bottom of convergence layer

        # Report problems finding the root first time it happens
        if (status_r != 0) and report_status_r:
            print("layer():")
            print(f" find_root(vfall) status = {status_r:3d} for {gas_name} at p = {p_layer/1e6:10.2e}")
            print("There may be more instances not reported")
            print("")
            print(f"status_r = {status_r:3d}")
            report_status_r = False
        
        if (status_r != 0) and report_status_q:
            print("layer():")
            print(f" find_root(advdiff) status = {status_q:3d} for {gas_name} at p = {p_layer/1e6:10.2e}")
            print(" there may be more instances not reported")
            print()
            print(f"status_q = {status_q}")
            report_status_q = False

        # Update properties at the bottom of next layer
        q_below = qt_top

        # Get layer averages

        if opd_layer > 0:
            reff_layer = 1.5 * qc_layer / (rho_p * opd_layer)
            lnsig2 = 0.5 * np.log( sig_layer )**2
            rg_layer = reff_layer * np.exp(-5 * lnsig2)
        else:
            reff_layer = 0.0
            rg_layer = 0.0

        qc_layer = qc_layer * gravity/dp_layer
        qt_layer = qt_layer * gravity/dp_layer

    return qc_layer, qt_layer, rg_layer, reff_layer, ndz_layer, opd_layer, kz_layer, report_status_r, report_status_q

# Implementation of eddysed.f
This is the main eddysed block of the code. Here we are finally get our desired output of q_c which is the condenste mass mixing ratio. Here a virtual layer is added to counteract the problem of convergence of the solution and if the condensable condenses before the periphery of the model grid.
This function calls the find_rootl function and pvap_gas function to be executed properly.

In [ ]:
def eddysed(gravity, T_eff, kz_min, cloudf_min, nsub_max, supsat, mw_atmos, do_virtual, num_layers, z_layers, z_layer_edges, p_layers, p_layer_edges, t_layers, t_layer_edges, chf, gas_name, gas_mmr, gas_mw, rho_p, sig, f_sed):
    
    """
    Main driver for EddySed cloud model
    Given atmospheric profiles and cloud parameters, calculate size and concentration of condensates in each layer of the atmosphere.

    Input Scalars :
    grav   :      gravitational acceleration (cm/s^2)
    teff    :     effective temperature (K)
    kz_min   :    minimum eddy diffusion coefficient (cm^2/s)
    cloudf_min :   minimum cloud fractional coverage
    nsub_max  :   maximum number of sublayers for adaptive mesh refinement
    supsat   :    supersaturation after condensation (fraction)
    mw_atmos   :  molecular weight of atmosphere (g/mol)
    do_virtual :  reduce mixing ratio due to decrease below cloud base
    nz    :       number of layers
    ngas  :       number of condensing gases

    
    Input Vectors:
    z     :       altitude at layer mid-pt (by pressure) (cm)
    z_top  :      altitude at top of layer (cm)
    p      :      pressure at layer mid-pt (by pressure) (dyne/cm^2)
    p_top  :      pressure at top of layer (dyne/cm^2)
    t     :       temperature at layer mid-pt (by pressure) (K)
    t_top  :      temperature at top of layer (K)
    chf     :     layer convective heat flux (erg/s/cm^2)
    gas_name  :   names of condensing gases
    gas_mmr  :    mass mixing ratio of gas below cloud base (g/g)
    gas_mw  :     molecular weight of gas (g/mol)
    rho_p   :     density of condensed vapor (g/cm^3)
    sig     :     geometric std deviation of lognormal size distribution
    f_sed :      ratio of microphysical sed flux to eddy sed flux

    Output Vectors:
    kz   :        eddy diffusion coefficient (cm^2/s)
    qt    :       total (gas+condensed) mixing ratio of condensate (g/g)
    qc :           mixing ratio of condensed condensate (g/g)
    ndz  :        number column density of condensate (cm^-3)
    rg  :          geometric mean radius of lognormal size distribution
    reff :        droplet effective radius (second moment of size distrib, cm)
    cloudf :      cloud fractional coverage (not applied to clouds)

    """

    # Initialization:
    cloud_base = 0
    report_status_r = True
    report_status_q = True

    # Calculate indices of bottom and top of domain
    if z_layers[1] > z_layers[0]:
        i_bottom = 0
        i_top = num_layers - 1
        k_bottom = 0
        k_top = num_layers - 1
        increment = 1
    else:
        i_bottom = num_layers - 1
        i_top = 0
        k_bottom = num_layers
        k_top = 1
        increment = -1

    # Start at the botto  of the domain: get p, T and qt
    t_bot = t_layer_edges[k_bottom]
    p_bot = p_layer_edges[k_bottom]
    q_below = gas_mmr

    # Initialize vertical path of condensate qc_path to zero
    qc_path = 0

    # Adjust mixing ratio at bottom of model domain if bottom layer is saturated
    if do_virtual:
        qvs_factor = (supsat+1) * gas_mw /mw_atmos
        pvap = pvap_gas(gas_name, t_bot, p_bot)
        qvs = qvs_factor * pvap / p_bot
        # eqn 3.62 from the book Atmospheric Survey by Walllace and Hobbs, the equation of q_vs to be alterred according to chemistry of the planet
        # e_s is the pvap function in the code which is taken from appendix A of AM01 eqn A1
        
        if qvs <= q_below:

            # Find pressure at cloud base

            # parameters for root finding
            p_low = p_bot
            p_high = p_bot * 100
            delta_q = q_below / 100

            # temperature gradient
            dtdlnp = ( t_layer_edges[k_bottom + increment] - t_bot)/ np.log( p_bot / p_layer_edges[k_bottom + increment] )

            # load parameters into qvs_below common block
            qv_dtdlnp = dtdlnp
            qv_p = p_bot
            qv_t = t_bot
            qv_gas_name = gas_name
            qv_factor = qvs_factor

            p_base, status_p = find_rootl(qvs_below, p_low, p_high, delta_q)

            if status_p != 0:
                print("Unable to find cloud base pressure in eddysed():")
                print(f"find_rootl(qvs_below) status = {status_p} for {gas_name}")

            t_base = t_bot + dtdlnp * np.log( p_bot / p_base )

            # Calculate temperature and pressure below bottom layer by adding a virtual layer
            p_layer = (p_bot + p_base) / 2
            t_layer = t_bot + dtdlnp * np.log( p_bot / p_layer )

            # Calculate qc, qt, rg, reff and ndz for virtual layer
            qc_layer, qt_layer, rg_layer, reff_layer, ndz_layer, opd_layer, kz_layer, report_status_r, report_status_q = layer(nsub_max, gas_name, kz_min, cloudf_min, mw_cloud=gas_mw, mw_atmos=mw_atmos, f_sed=f_sed, rho_p=rho_p, supsat=supsat, sig_layer=sig, t_layer=t_layer, p_layer=p_layer, t_top=t_base, t_bot=t_bot, p_top=p_base, p_bot=p_bot, chf_layer=chf[i_bottom], gravity=gravity, q_below=q_below, report_status_r=report_status_r, report_status_q=report_status_q)

            # report optical depth below domain
            print(f"eddysed(): Condensing gas = {gas_name}")
            print(f"cloud base at p, T = {p_base/1e6:10.2e}, {t_base:8.2f}")
            opd_belowdomain = 1.5 * qc_layer * (p_base - p_bot) / gravity / (rho_p * reff_layer)
            print(f"optical depth below domain = {opd_belowdomain:10.4e}")

    qc = np.zeros(num_layers)
    qt = np.zeros(num_layers)
    rg = np.zeros(num_layers)
    reff = np.zeros(num_layers)
    ndz = np.zeros(num_layers)
    kz = np.zeros(num_layers)

    # Loop over atmospheric layers from the bottom up
    for i_layer in range(i_bottom, i_top + increment, increment):
        if increment == 1:
            k_layer = i_layer
        else:
            k_layer = i_layer + 1
        
        qc_layer, qt_layer, rg_layer, reff_layer, ndz_layer, opd_layer, kz_layer, report_status_r, report_status_q = layer(nsub_max, gas_name, kz_min, cloudf_min, mw_cloud=gas_mw, mw_atmos=mw_atmos, f_sed=f_sed, rho_p=rho_p, supsat=supsat, sig_layer=sig[i_layer], t_layer=t_layers[i_layer], p_layer=p_layers[i_layer], t_top=t_layer_edges[k_layer + increment], t_bot=t_layer_edges[k_layer], p_top=p_layer_edges[k_layer + increment], p_bot=p_layer_edges[k_layer], chf_layer=chf[i_layer], gravity=gravity, q_below=q_below, report_status_r=report_status_r, report_status_q=report_status_q)

        qc[i_layer] = qc_layer
        qt[i_layer] = qt_layer
        rg[i_layer] = rg_layer
        reff[i_layer] = reff_layer
        ndz[i_layer] = ndz_layer
        kz[i_layer] = kz_layer

        # Accumulate vertical path of condensate
        qc_path += qc_layer * ( p_layer_edges[k_layer] - p_layer_edges[k_layer + increment] ) / gravity

    # Print some diagnostics
    print(f"eddysed(): condensing gas = {gas_name}")
    print(f"condensate path = {qc_path*1e4} g/m^2")

    return kz, qt, qc, ndz, rg, reff